In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import nn
import collections
from rdkit.Chem import Descriptors
from rdkit import Chem
#from tensorflow.keras.mixed_precision import LossScaleOptimizer
#policy = LossScaleOptimizer('mixed_float16')
#mixed_precision.set_policy(policy)
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix, mean_squared_error
import statsmodels.api as sm

In [ ]:
data = pd.read_csv('ratio_dataset.csv', sep="\t",low_memory=False)

In [ ]:
#params
#for quick prototyping epoch count is set to 20
epoch_count = 20

In [ ]:
#Generate morgan FP, may take a while
bins_len= 512
smiles_train = data["RP"]
def vectorize(smiles):
    x_one = np.zeros((len(data), 8, bins_len ),dtype=np.int8)
    #x_one = []
    for ca,smil in enumerate(smiles):
        #x_one_ = []
        qq = list(smil.split("."))
        for fa in range(0,8):
            try:
                x_one[ca][fa]=np.fromiter((map(int, AllChem.GetMorganFingerprintAsBitVect(AllChem.MolFromSmiles(qq[fa]),3,nBits=bins_len).ToBitString())),int)
            except:
                pass
            #x_one_.append([0 for x in range(0,bins_len)])
            #print(smil)
        
    return x_one
X_train = vectorize(smiles_train.values)

In [ ]:
#saving a numpy array is recommended, to avoid generating morgan FP every time
#np.save('X_train_rat.npy', X_train)
# npy file can be loaded with:
#X_train = np.load("X_train_rat.npy")

In [ ]:
X_train_flat = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))

In [ ]:
# vectorize output
one_hot_yz = np.zeros((len(data), 1 ),dtype=np.float16)
for i,nam in enumerate(data['rat2'])
    one_hot_yz[i] = float(nam)          
Y_train = np.array(one_hot_yz)


In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,X_train_flat, test_size=0.1,shuffle=True)

In [ ]:
######## LSTM ~~~~~~~~~~~~~~
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(8,512,))
# "encoded" is the encoded representation of the input
encoded = layers.LSTM(encoding_dim)(input_img)
# "decoded" is the lossy reconstruction of the input

decoded = layers.Dense(4096, activation='sigmoid')(encoded)
#resh = layers.Reshape((8,512))(decoded)
# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
#encoder = keras.Model(input_img, encoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(8,512,))


enc2 = autoencoder.layers[1](input_img)

#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(256, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(ls)
p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='mse', optimizer="adam")
pred_model.layers[1].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train_flat,X_train_flat, test_size=0.1,shuffle=True)

In [ ]:
######## Dense ~~~~~~~~~~~~~~
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(4096,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim,activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input

decoded = layers.Dense(4096, activation='sigmoid')(encoded)
#resh = layers.Reshape((8,512))(encoded)
# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=20,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(4096,))

#input_to_model = encoder
#x = layers.Dense(32,activation='relu')(input_to_model)
'''
enc2 = encoder.layers[-8](input_img)
enc2 = encoder.layers[-7](enc2)
enc2 = encoder.layers[-6](enc2)
enc2 = encoder.layers[-5](enc2)

enc2 = encoder.layers[-4](enc2)
enc2 = encoder.layers[-3](enc2)


enc2 = encoder.layers[-2](enc2)
'''
enc2 = autoencoder.layers[1](input_img)

#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(256, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(ls)
p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='mse', optimizer="adam")
pred_model.layers[1].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train_flat,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,X_train, test_size=0.1,shuffle=True)

In [ ]:
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(8,512,))
# "encoded" is the encoded representation of the input
a = layers.Permute((2,1))(input_img)
x = layers.Conv1D(256,12,activation="relu")(a)
x1 = layers.MaxPooling1D(2)(x)
x2 = layers.Conv1D(128,10, activation='relu')(x1)
x3 = layers.MaxPooling1D(2)(x2)
x3 = layers.Conv1D(64,10, activation='relu')(x3)
x4 = layers.AveragePooling1D()(x3)
flat = layers.Flatten()(x4)
encoded = layers.Dense(encoding_dim)(flat)
d1 = layers.Dense(64)(encoded)
d2 = layers.Reshape((16,4))(d1)
d3 = layers.Conv1D(256,1,strides=1, activation='relu')(d2)
d4 = layers.UpSampling1D(2)(d3)
d4 = layers.UpSampling1D(2)(d4)
d5 = layers.Conv1D(256,1,strides=1, activation='relu')(d4)
d6 = layers.UpSampling1D(2)(d5)
d6 = layers.UpSampling1D(2)(d6)
d6 = layers.Conv1D(256,1,strides=1, activation='relu')(d6)
d7 =layers. UpSampling1D(2)(d6)
decoded = layers.Conv1D(8,1,strides=1, activation='sigmoid')(d7)
decoded = layers.Permute((2,1))(decoded)
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(8,512,))

#input_to_model = encoder
#x = layers.Dense(32,activation='relu')(input_to_model)

enc2 = autoencoder.layers[1](input_img)
enc2 = autoencoder.layers[2](enc2)
enc2 = autoencoder.layers[3](enc2)
enc2 = autoencoder.layers[4](enc2)

enc2 = autoencoder.layers[5](enc2)
enc2 = autoencoder.layers[6](enc2)


enc2 = autoencoder.layers[7](enc2)
enc2 = autoencoder.layers[8](enc2)
enc2 = autoencoder.layers[9](enc2)
#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(enc2)
ls = layers.Dense(64, activation="relu")(ls)
p = layers.Dense(1, activation="linear",dtype='float32',name="ratio")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='mse', optimizer="adam")
pred_model.layers[1].trainable=False
pred_model.layers[2].trainable=False
pred_model.layers[3].trainable=False
pred_model.layers[4].trainable=False
pred_model.layers[5].trainable=False
pred_model.layers[6].trainable=False
pred_model.layers[7].trainable=False
pred_model.layers[8].trainable=False
pred_model.layers[9].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)

In [ ]:
y_pred2=modelx.predict(X_test2,verbose=1)

print('MSE: ',mean_squared_error([float(x) for x in y_test2_2], [float(x) for x in y_pred2]))

#plt.axis([0, 1.1, 0, 1.1])
#plt.scatter((y_pred2),(y_test2_2))
print('Correlation cof matrix:')
print(np.corrcoef([float(x) for x in y_pred2], [float(x) for x in y_test2_2]))
X_addC = sm.add_constant([float(x) for x in y_test2_2])
result = sm.OLS([float(x) for x in y_pred2], X_addC).fit()
print('R2:',result.rsquared,'R2 adj:', result.rsquared_adj)